In [18]:
!pip install anthropic
!pip install openai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.6/948.6 kB 17.8 MB/s eta 0:00:00a 0:00:01


In [2]:
!pip install openpyxl

In [ ]:
from pathlib import Path
import pandas as pd
from openai import OpenAI
import re


client = OpenAI(api_key=openai_api_key)


In [42]:

def complete_with_continuation(prompt, model="gpt-5-nano", save_path = "", max_tokens=4096):
    messages = [{"role": "user", "content": prompt}]
    full_output = ""
    total_input_tokens = 0
    total_cached_tokens = 0
    total_output_tokens = 0

    while True:
        response = client.chat.completions.create(
            model=model,
            # max_tokens=max_tokens,
            messages=messages
        )

        choice = response.choices[0]
        content = choice.message.content   # ✅ fixed
        finish_reason = choice.finish_reason

        # Add the new output to the full string
        full_output += content

        # --- Token + cost tracking ---
        usage = response.usage
        input_tokens = usage.prompt_tokens
        cached_input_tokens = getattr(usage, "cached_prompt_tokens", 0)  # safe fallback
        output_tokens = usage.completion_tokens
        total_tokens = usage.total_tokens

        # Accumulate totals
        total_input_tokens += input_tokens
        total_cached_tokens += cached_input_tokens
        total_output_tokens += output_tokens

        # print(f"\n--- API Call Usage ---")
        # print(f"Input tokens: {input_tokens}")
        # print(f"Cached input tokens: {cached_input_tokens}")
        # print(f"Output tokens: {output_tokens}")
        # print(f"Total tokens: {total_tokens}")

        # Cost for this call
        input_cost = input_tokens * (0.05 / 1_000_000)
        cached_input_cost = cached_input_tokens * (0.005 / 1_000_000)
        output_cost = output_tokens * (0.40 / 1_000_000)
        call_cost = input_cost + cached_input_cost + output_cost
        # print(f"Estimated cost (this call): ${call_cost:.6f}")

        if finish_reason == "length":
            # Truncated — add a continuation message
            messages.append({"role": "assistant", "content": content})
            messages.append({"role": "user", "content": "Continue where you left off."})
        else:
            break

    # --- Final total cost ---
    total_cost = (
        total_input_tokens * (0.05 / 1_000_000)
        + total_cached_tokens * (0.005 / 1_000_000)
        + total_output_tokens * (0.40 / 1_000_000)
    )

    # print(f"\n=== Total Usage Across All Calls ===")
    # print(f"Input tokens: {total_input_tokens}")
    # print(f"Cached input tokens: {total_cached_tokens}")
    # print(f"Output tokens: {total_output_tokens}")
    # print(f"Grand total tokens: {total_input_tokens + total_cached_tokens + total_output_tokens}")
    # print(f"Estimated total cost: ${total_cost:.6f}")
    # Prepare a dictionary with the info
    usage_data = {
        "Metric": [
            "Input tokens",
            "Cached input tokens",
            "Output tokens",
            "Grand total tokens",
            "Estimated total cost"
        ],
        "Value": [
            total_input_tokens,
            total_cached_tokens,
            total_output_tokens,
            total_input_tokens + total_cached_tokens + total_output_tokens,
            total_cost
        ]
    }

    # Create a DataFrame
    df_usage = pd.DataFrame(usage_data)

    # Ensure parent directories exist
    save_path.parent.mkdir(parents=True, exist_ok=True)

    # Save to Excel
    df_usage.to_excel(save_path, index=False)

    return full_output


In [36]:

    
# 60 questions
# read the Excel file, first sheet
df_60 = pd.read_excel("60questions.xlsx", sheet_name=0)

# get all id and question column questions in a list, id followed by question
# 
# make list into \n separated, str 

combined_list = df_60.apply(lambda row: f"{row['id']}: {row['question']}", axis=1).tolist()

# Join into one string with newline separators
questions_block = "\n".join(combined_list)

print(questions_block)


1101: Does the paper report previously unpublished data?
1102: Does the paper report HIV sequences?
1103: Does the paper report the results of in vitro passage experiments?
1104: Does the paper report in vitro antiretroviral susceptibility data?
2101: Does the paper report GenBank accession numbers for sequenced HIV isolates?
2102: Does the paper report GenBank accession numbers for sequenced HIV isolates other than those for laboratory HIV isolates?
2103: Which are the genbank accession numbers reported in the paper?
2202: Does the paper report lists of mutations for individual sequenced HIV isolates?
2301: Which HIV species were studied in the paper?
2302: What were the subtypes of the sequenced viruses reported in the paper?
2303: Which HIV genes were sequenced in the paper?
2304: Does the paper report the results of HIV pol sequences?
2401: Which geographic regions and/or countries were the sequences from in the paper?
2402: What years were the sequenced samples obtained in the pap

In [ ]:
# gpt-4o

model = "gpt-5"
file_path = Path("papers 250") / "60"
rows = []

with open("explain_multi_questions.txt", "r", encoding="utf-8") as f:
    template = f.read()

tmp_content = None
tmp_name = None
for folder in file_path.iterdir():
    if folder.is_dir():
        doc_folder = folder / "document"
        if doc_folder.exists():
            for md_file in doc_folder.glob("*.checked.md"):
                with md_file.open(encoding="utf-8") as f:
                    content = f.read()
                    #test 1 first
                    tmp_name = md_file.stem.replace(".checked", "")  # Remove the .checked part
                    tmp_content = content

                    output_file = Path(f"Results/{model}/{tmp_name}/60_questions_answer.txt")
                
                    # Skip if output file already exists
                    if output_file.exists():
                        print(f"Skipping {tmp_name}, output file already exists.")
                        continue

                    

                    prompt_60 = template.format(
                        paper_content=tmp_content,
                        question=questions_block.strip()
                    )

                    # --- Step 6: Save final prompt ---
                    # output_file = Path(f"Results/{model}/{tmp_name}/_60_questions_prompt.txt")
                    # with output_file.open("w", encoding="utf-8") as f:
                    #     f.write(prompt_60)
                    # print(prompt_60)

                    
                    answer = complete_with_continuation(prompt_60, model=model, save_path = Path(f"Results/{model}/{tmp_name}/cost.xlsx"), max_tokens=4096)

                    print(answer)
                    output_file = Path(f"Results/{model}/{tmp_name}/60_questions_answer.txt")


                    # Create the parent directories if they don't exist
                    output_file.parent.mkdir(parents=True, exist_ok=True)
                    with output_file.open("w", encoding="utf-8") as f:
                        f.write(answer)
                    

Skipping Yao2015, 25410050, output file already exists.
"""
Question: 1101 

Evidence: A cross-sectional study design was used to study the DR profiles in LLV patients who were followed up in Jiangsu Province in 2021. In this study, a total of 345 samples were collected throughout 2021.

Rationale: The paper describes a new cross-sectional investigation conducted in 2021 with newly collected samples and analyses, indicating that it reports previously unpublished data from this cohort.

Answer: Yes
"""

"""
Question: 1102 

Evidence: After nucleic acid extraction, PCR amplification and sequencing were performed on the plasma of 345 samples, Pol genes of 255 patients were obtained. Thirteen cases of sequences with poor quality or unmatched information were removed, and 242 cases of sequences meeting the research requirements were finally included.

Rationale: The text explicitly states that HIV pol genes were amplified and sequenced from patient plasma, confirming that the study reports 

In [ ]:




pattern = r"Question:\s*(.*?)\s*Evidence:\s*(.*?)\s*Rationale:\s*(.*?)\s*Answer:\s*(.*?)(?=\n---|\nQuestion:|$)"


base_dir = Path(f"Results/{model}")

# Loop over every folder in base_dir
for tmp_folder in base_dir.iterdir():
    if tmp_folder.is_dir():
        file_path = tmp_folder / "60_questions_answer.txt"
        if file_path.exists():
            # Read the file
            with open(file_path, "r") as f:
                content = f.read()
            

    # for text file in output_file = Path(f"Results/{model}/{tmp_name}/60_questions_answer.txt")

    matches = re.findall(pattern, content, re.DOTALL)

    # --- Step 3: Build dataframe ---
    records = []

    for i, (q, ev, rat, ans) in enumerate(matches):
        # strip extra whitespace
        q, ev, rat, ans = q.strip(), ev.strip(), rat.strip(), ans.strip()

        # Clean the answer
        cleaned_ans = ans.replace("\n", " ")         # remove line breaks
        cleaned_ans = re.sub(r'"+', '', cleaned_ans) # remove repeated quotes
        cleaned_ans = re.sub(r'\s+', ' ', cleaned_ans).strip()  # collapse multiple spaces
        
        # match to QID + original Question
        if i < len(combined_list):
            id_str, question_str = combined_list[i].split(": ", 1)
            records.append({
                "PMID": tmp_name,
                "QID": id_str,
                "Question": question_str,
                "Evidence": ev,
                "Rationale": rat,
                "Answer": cleaned_ans
            })

    output_df = pd.DataFrame(records)

    # --- Step 4: Save to Excel ---
    output_df.to_excel(f"Results/{model}/{tmp_folder}/gpt_answers_60.xlsx", index=False)








--- API Call Usage ---
Input tokens: 6436
Cached input tokens: 0
Output tokens: 14692
Total tokens: 21128
Estimated cost (this call): $0.006199

=== Total Usage Across All Calls ===
Input tokens: 6436
Cached input tokens: 0
Output tokens: 14692
Grand total tokens: 21128
Estimated total cost: $0.006199
"""
Question: 1101 

Evidence: From December 2017 through May 2018, a total of 276 youth age 15 and 24 years from four clinics were recruited into the parent Project YES! study. A cross-sectional analysis of all HIVDR results at study enrollment was done to determine HIVDR prevalence and characteristics among adolescents and young adults with virologic treatment failure.

Rationale: The study presents newly collected enrollment data and conducts a cross-sectional analysis of HIV drug resistance results, indicating the data are from a new, unpublished study.

Answer: Yes
"""

"""
Question: 1102 

Evidence: HIV viral load was determined using the Cobas Ampliprep/CobasTaqman 96 machine. Blo

## Claude results

In [18]:
from anthropic import Anthropic
client = Anthropic(api_key=claude_api_key)

response = client.messages.create(
    model="claude-3-haiku-20240307", # claude-3-haiku-20240307
    max_tokens=4096,
    messages=[
        {"role": "user", "content": prompt_60}
    ]
)

answer = response.content[0].text

# --- Step 1: Token usage ---
input_tokens = response.usage.input_tokens
output_tokens = response.usage.output_tokens
total_tokens = input_tokens + output_tokens

print(f"Input tokens: {input_tokens}")
print(f"Output tokens: {output_tokens}")
print(f"Total tokens: {total_tokens}")

# --- Step 2: Pricing (Claude Opus 4.1 as of 2025-09) ---
# Input: $15.00 per 1M tokens = $0.000015 per token
# Output: $75.00 per 1M tokens = $0.000075 per token
input_cost = input_tokens * 0.000015
output_cost = output_tokens * 0.000075
total_cost = input_cost + output_cost

print(f"Estimated cost: ${total_cost:.4f}")


Input tokens: 5262
Output tokens: 3866
Total tokens: 9128
Estimated cost: $0.3689


Here are the answers to the questions:

Question: 1101

Evidence: The paper states that "The epidemic characteristics of human immunodeficiency virus type 1 (HIV-1) in Zhejiang Province have not been systematically identified." This suggests that the data reported in the paper are previously unpublished.

Rationale: The paper presents new data on the epidemic characteristics and molecular epidemiology of HIV-1 in Zhejiang Province, China, which has not been systematically studied before.

Answer: Yes

Question: 1102

Evidence: The paper states that "Totally, 332 *gag* and 229 *pol* gene fragments were amplified and sequenced from the 451 individual samples, respectively."

Rationale: The paper reports the sequencing of HIV-1 gag and pol gene fragments from patient samples.

Answer: Yes

Question: 1103 

Evidence: There is no mention of in vitro passage experiments in the paper.

Rationale: The paper does not report any results from in vitro passage experiments.

Answer: No

Question: 1